In [1]:
from bs4 import BeautifulSoup as bs
import requests 
import pymongo
import pandas as pd
import re
import os
import csv
import time
import json
import ast
import pprint
from datetime import datetime, timedelta

In [ ]:
def process_all_files(corona_file):

    corona_url = "https://github.com/globalcitizen/2019-wuhan-coronavirus-data/blob/master/" + corona_file
    print("now processing: " + corona_url)

    new_date = corona_file[22:26]+"-"+corona_file[26:28]+"-"+corona_file[28:30]
    print(new_date)

    response = requests.get(corona_url)
    time.sleep(10)
    print(response)
    soup = bs(response.text, 'lxml')
    my_text = soup.find_all('td', class_='blob-code blob-code-inner js-file-line', attrs={"id":"LC1"})

#     print(my_text)
    limit = 100

    if response.status_code == 200:
        for index, d_fl in enumerate(my_text):
        #     print(d_fl.text)
            my_scrape_file = d_fl.text
            if index==limit:
                break

#         print(my_scrape_file)        

        my_series = eval(my_scrape_file)
       
        my_scrape_df = pd.DataFrame(my_series)
        
        provinces = pd.read_csv("static/master_data/provinces.csv")
        df_provinces = pd.DataFrame(provinces).set_index("provinceShortName")
#         df_provinces
#         df_data = pd.read_csv("data/df_2020-01-26.csv")
        my_scrape_df = pd.merge(df_provinces, my_scrape_df, on=("provinceShortName"))
        my_scrape_df = my_scrape_df[['american_name','provinceName',"provinceShortName", "confirmedCount", "suspectedCount",'curedCount','deadCount']]
        my_scrape_df
        
#         my_scrape_df = my_scrape_df[['provinceName', "provinceShortName", "confirmedCount", "suspectedCount",'curedCount','deadCount', 'cities']]
        my_scrape_df['date'] = new_date
        output_path = os.path.join("static/data", "df_" + new_date + ".csv")
        my_scrape_df.to_csv(output_path)
#         my_scrape_df.head()

In [ ]:
input_file = open('static/data/urls_to_data_files.txt')
try:
    for i, line in enumerate(input_file):
        # Remove hard return from line-string
        line = line[:-1]
        print("---"+line+"---")
        corona_file = line
        process_all_files(corona_file)
        print(line)
finally:
    input_file.close()
    print("Done")
# print f"{0} line(s) printed".format(i+1)

In [ ]:
def create_geojson(date, filename):
#     df_data = pd.read_csv("static/data/df_" + date + ".csv")
    df_data = pd.read_csv("static/data/" + filename)

    # df_json = (df_data).to_json()
    df_data = df_data.set_index("provinceName")
#     df_data

    json_df = pd.read_json('static/geojsons/china.json', encoding='UTF-8')
    json_df_feat = pd.DataFrame(json_df.features)

#     with open('static/geojsons/china2.json', encoding='UTF-8') as f:
#       china_json = json.load(f)
# #     pprint.pprint(china_json)

    geo_dict = {}
    geo_string = ""

    for index, row in json_df_feat.iterrows():
        try:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['name']

            prop_american_name = df_data.loc[prop_name,"american_name"]
            prop_confirmedCount = df_data.loc[prop_name,"confirmedCount"]
            prop_suspectedCount = df_data.loc[prop_name,"suspectedCount"]
            prop_curedCount = df_data.loc[prop_name,"curedCount"]
            prop_deadCount = df_data.loc[prop_name,"deadCount"]    
            prop_date = df_data.loc[prop_name,"date"]

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
        #     print(str_prop_all)
        #     print('=============================')    
        #     print(row['features']['geometry'])
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")
        #     print(str_geom_1)
        #     print('=============================')

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
        #     print(str_for_each_province)
            geo_string = geo_string + (str_for_each_province)
        except:
            str_feat_1 = ("{\"type\": \"Feature\",")

            prop_name = row['features']['properties']['name']

            prop_american_name = ""
            prop_confirmedCount = 0
            prop_suspectedCount = 0
            prop_curedCount = 0
            prop_deadCount = 0    
            prop_date = date

            str_prop_double_quotes = str(row['features']['properties'])
            str_prop_double_quotes = str_prop_double_quotes.replace("\'","\"")

            str_prop_1 = ("\"properties\" : " + str_prop_double_quotes + "\",")[:-3]
            str_prop_2 = (",\"american_name\" : \"" + prop_american_name + "\",")
            str_prop_3 = ("\"confirmedCount\" : \"" + str(prop_confirmedCount) + "\",")
            str_prop_4 = ("\"suspectedCount\" : \"" + str(prop_suspectedCount) + "\",")
            str_prop_5 = ("\"curedCount\" : \"" + str(prop_curedCount) + "\",")
            str_prop_6 = ("\"deadCount\" : \"" + str(prop_deadCount) + "\",")
            str_prop_7 = ("\"date\" : \"" + prop_date + "\"},")

            str_prop_all  = str_prop_1 + str_prop_2 + str_prop_3 + str_prop_4 + str_prop_5 + str_prop_6 + str_prop_7
        #     print(str_prop_all)
        #     print('=============================')    
        #     print(row['features']['geometry'])
            str_geom_1 = ("\"geometry\":" + str(row['features']['geometry']) + "},")
            str_geom_1 = str_geom_1.replace("\'","\"")
        #     print(str_geom_1)
        #     print('=============================')

            str_for_each_province = (str_feat_1)+(str_prop_all)+(str_geom_1)
        #     print(str_for_each_province)
            geo_string = geo_string + (str_for_each_province)
    pre_fix = "{\"type\": \"FeatureCollection\", \"features\": ["
    post_fix = "]}"

    total_string = pre_fix + geo_string[:-1] + post_fix

    print(total_string)
    output_path = os.path.join("static/geojsons", date + ".json")
    with open(output_path, "w", encoding='UTF-8') as text_file:
        text_file.write(total_string)
        text_file.close()

In [ ]:
# For later. Loop through all the files created
for filename in os.listdir("static/data"):
    if filename.endswith(".csv"):
         # print(os.path.join(directory, filename))
        print(filename)
        date = filename[3:13]
        print(date)
        create_geojson(date, filename)
        continue
    else:
        continue

In [26]:
# Get the data from 3 different data source
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df_confirmed = pd.read_csv(confirmed_url)
cured_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
df_cured = pd.read_csv(cured_url)
death_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
df_death = pd.read_csv(death_url)
# total_columns = len(df_death.columns)
# df_death.head()
# tables.describe()
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20
0,Anhui,Mainland China,31.82571,117.2264,1,9,15,39,60,70,...,973,982,986,987,988,989,989,989,989,989
1,Beijing,Mainland China,40.18238,116.4142,14,22,36,41,68,80,...,381,387,393,395,396,399,399,399,400,400
2,Chongqing,Mainland China,30.05718,107.8740,6,9,27,57,75,110,...,553,555,560,567,572,573,575,576,576,576
3,Fujian,Mainland China,26.07783,117.9895,1,5,10,18,35,59,...,290,292,293,293,293,293,293,293,294,294
4,Gansu,Mainland China,36.06110,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,91


In [27]:
# Create unique rows by moving date column to row level for all Confirmed cases
df_confirmed_long = df_confirmed.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_confirmed_long.head()
output_path = os.path.join("static/world_data", "df_world_confirmed_original.csv")
df_confirmed.to_csv(output_path)

# Create unique rows by moving date column to row level for all Cured cases
df_cured_long = df_cured.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_cured_long.head()
output_path = os.path.join("static/world_data", "df_world_cured_original.csv")
df_cured.to_csv(output_path)

# Create unique rows by moving date column to row level for all Death cases
df_death_long = df_death.melt(id_vars=["Province/State", "Country/Region","Lat","Long"], 
        var_name="Date", 
        value_name="Value")
df_death_long.head()
output_path = os.path.join("static/world_data", "df_world_death_original.csv")
df_death.to_csv(output_path)

df_confirmed_long.head()
df_confirmed_long["new_date"] = pd.to_datetime(df_confirmed_long['Date'])
# df_confirmed_long.drop(['Date'], axis=1)

# df_temp=df_death_long[df_death_long["Province/State"].isnull()]
# df_temp.head()
df_confirmed_long.head()

,Province/State,Country/Region,Lat,Long,Date,Value,new_date
0,Anhui,Mainland China,31.82571,117.2264,1/22/20,1,2020-01-22
1,Beijing,Mainland China,40.18238,116.4142,1/22/20,14,2020-01-22
2,Chongqing,Mainland China,30.05718,107.8740,1/22/20,6,2020-01-22
3,Fujian,Mainland China,26.07783,117.9895,1/22/20,1,2020-01-22
4,Gansu,Mainland China,36.06110,103.8343,1/22/20,0,2020-01-22


In [28]:
# If Province/State isn't filled, use the country value instead

df_confirmed_long["Province/State"].fillna(df_confirmed_long["Country/Region"], inplace=True)
df_confirmed_long = df_confirmed_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long", "Date":"date", "Value": "conf_count"})

df_cured_long["Province/State"].fillna(df_cured_long["Country/Region"], inplace=True)
df_cured_long = df_cured_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "cured_count"})

df_death_long["Province/State"].fillna(df_death_long["Country/Region"], inplace=True)
df_death_long = df_death_long.rename(columns={"Province/State":"american_name", "Country/Region":"country", "Lat":"lat","Long":"long","Date":"date", "Value": "dead_count"})


In [29]:

df_confirmed_long.loc[(df_confirmed_long.country == 'Mainland China'),'country']='China'
df_confirmed_long = df_confirmed_long.set_index(['date', 'american_name'])

df_cured_long.loc[(df_cured_long.country == 'Mainland China'),'country']='China'
df_cured_long = df_cured_long.set_index(['date', 'american_name'])

df_death_long.loc[(df_death_long.country == 'Mainland China'),'country']='China'
df_death_long = df_death_long.set_index(['date', 'american_name'])


In [30]:

df_confirmed_long

country       lat      long  conf_count  \
date    american_name                                                      
1/22/20 Anhui                      China  31.82571  117.2264           1   
        Beijing                    China  40.18238  116.4142          14   
        Chongqing                  China  30.05718  107.8740           6   
        Fujian                     China  26.07783  117.9895           1   
        Gansu                      China  36.06110  103.8343           0   
...                                  ...       ...       ...         ...   
2/26/20 Georgia                  Georgia  42.31540   43.3569           1   
        Greece                    Greece  39.07420   21.8243           1   
        North Macedonia  North Macedonia  41.60860   21.7453           1   
        Norway                    Norway  60.47200    8.4689           1   
        Romania                  Romania  45.94320   24.9668           1   

                          new_date  
date    american_name               
1/22/20 Anhui           2020-01-22  
        Beijing         2020-01-22  
        Chongqing       2020-01-22  
        Fujian          2020-01-22  
        Gansu           2020-01-22  
...                            ...  
2/26/20 Georgia         2020-02-26  
        Greece          2020-02-26  
        North Macedonia 2020-02-26  
        Norway          2020-02-26  
        Romania         2020-02-26  

[3636 rows x 5 columns]

In [34]:

df_merged = pd.merge(df_confirmed_long, df_cured_long, how='left', on=['date', 'american_name', 'country','lat','long'])

df_merged = pd.merge(df_merged, df_death_long,how='left', on=['date', 'american_name', 'country','lat','long'])
df_merged = df_merged.reset_index()
df_merged = df_merged.drop(['date'], axis=1)
df_merged = df_merged.rename(columns={"new_date":"date"})

output_path = os.path.join("static/world_data", "df_world_all.csv")
df_merged.to_csv(output_path)


df_merged.head()

,american_name,country,lat,long,conf_count,date,cured_count,dead_count
0,Anhui,China,31.82571,117.2264,1,2020-01-22,0,0
1,Beijing,China,40.18238,116.4142,14,2020-01-22,0,0
2,Chongqing,China,30.05718,107.8740,6,2020-01-22,0,0
3,Fujian,China,26.07783,117.9895,1,2020-01-22,0,0
4,Gansu,China,36.06110,103.8343,0,2020-01-22,0,0
